# Kapitola 36: Rizika AI - Kdyz se dobry sluha stane zlym panem

V teto kapitole se podivame na temnou stranu umele inteligence: algoritmickou nespravedlnost (bias), rizika zneuziti a jak provadet **audit fairness** nasich modelu.

## Co se naucime:
- Hlavni rizika AI (soukromi, autonomie, nespravedlnost)
- Co je algoritmicky bias a jak vznika
- Prakticky audit spravedlnosti na realnych datech
- Metriky fairness a jak je interpretovat

---

## 1. Instalace a import knihoven

In [ ]:
# Instalace potrebnych knihoven
!pip install pandas scikit-learn seaborn matplotlib numpy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Nastaveni
plt.rcParams['figure.figsize'] = [12, 6]
sns.set_style('whitegrid')

print("Knihovny nacteny!")
print()
print("V teto kapitole se naucime:")
print("1. Jaká jsou hlavni rizika AI")
print("2. Co je algoritmicky bias")
print("3. Jak provadet audit spravedlnosti")

---

## 2. Tri hlavni rizika umele inteligence

AI prinasi obrovske moznosti, ale take vyznamna rizika:

In [ ]:
# Vizualizace tri hlavnich rizik AI

rizika = {
    'Zneuziti dat\na ztrata soukromi': [
        'Digitalni profily bez souhlasu',
        'Cilena manipulace',
        'Odhaleni citlivych informaci'
    ],
    'Autonomni systemy\na eticka dilemata': [
        'Rozhodnuti samoridiciho auta',
        'Kdo je zodpovedny za chybu?',
        'Vojenske vyuziti AI'
    ],
    'Algoritmicka\nnespravedlnost (Bias)': [
        'Diskriminace pri prijimani',
        'Nespravedlive uvery',
        'Predpojatost ve zdravotnictvi'
    ]
}

fig, axes = plt.subplots(1, 3, figsize=(16, 6))
barvy = ['#ff6b6b', '#ffd93d', '#6bcb77']

for ax, (nazev, priklady), barva in zip(axes, rizika.items(), barvy):
    ax.add_patch(plt.Rectangle((0, 0), 1, 1, facecolor=barva, alpha=0.3))
    ax.text(0.5, 0.85, nazev, ha='center', va='center', fontsize=13, fontweight='bold')
    for i, priklad in enumerate(priklady):
        ax.text(0.5, 0.55 - i*0.2, f'• {priklad}', ha='center', va='center', fontsize=10)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')

plt.suptitle('Tri hlavni rizika umele inteligence', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Detailni vysvetleni
print("=" * 70)
print("ALGORITMICKA NESPRAVEDLNOST (BIAS) - NEJZAKLADNEJSI PROBLEM")
print("=" * 70)
print()
print("Jak bias vznika?")
print("-" * 50)
print("1. HISTORICKA DATA: Model se uci z minulosti")
print("   - V minulosti bylo mene zen na manaerskych pozicich")
print("   - Model se 'nauci', ze zeny jsou horsi kandidatky")
print()
print("2. NEVYVAZENA DATA: Nerovnomerne zastoupeni skupin")
print("   - Malo dat o minoritach -> horsi predikce pro ne")
print()
print("3. CHYBNE LABELY: Predpojati anotatori")
print("   - Lidske predsudky se prenesou do dat")
print()
print("4. PROXY PROMENNE: Neprime diskriminacni znaky")
print("   - PSC muze korelovat s rasou")
print("   - Model diskriminuje, aniz by primo pouzival rasu")

---

## 3. Dataset Adult - Predikce prijmu

Pouzijeme slavny dataset **Adult** (Census Income) k demonstraci algoritmicke nespravedlnosti. Cilem je predpovedet, zda osoba vydela vice nez $50K rocne.

In [ ]:
# Nacteni Adult datasetu
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
           'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
           'hours-per-week', 'native-country', 'income']

try:
    df = pd.read_csv(url, header=None, names=columns, na_values=' ?', sep=', ', engine='python')
    print("Data uspesne nactena z UCI repository!")
except:
    # Fallback - vytvorime synteticka data
    print("Vytvarim synteticka data pro demonstraci...")
    np.random.seed(42)
    n = 5000
    df = pd.DataFrame({
        'age': np.random.randint(18, 70, n),
        'education-num': np.random.randint(1, 16, n),
        'hours-per-week': np.random.randint(10, 80, n),
        'sex': np.random.choice(['Male', 'Female'], n, p=[0.65, 0.35]),
        'race': np.random.choice(['White', 'Black', 'Asian', 'Other'], n, p=[0.7, 0.15, 0.1, 0.05]),
        'income': np.random.choice(['<=50K', '>50K'], n, p=[0.75, 0.25])
    })

print(f"\nPocet zaznamu: {len(df)}")
print(f"Pocet sloupcu: {len(df.columns)}")
df.head()

In [ ]:
# Cisteni dat
df_clean = df.dropna().copy()
print(f"Zaznamu po odstraneni chybejicich hodnot: {len(df_clean)}")

# Vizualizace rozdeleni prijmu podle pohlavi
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Rozdeleni prijmu
income_counts = df_clean['income'].value_counts()
axes[0].pie(income_counts.values, labels=income_counts.index, autopct='%1.1f%%', 
            colors=['#ff9999', '#99ff99'], startangle=90)
axes[0].set_title('Rozdeleni prijmu v datasetu', fontsize=14)

# Rozdeleni podle pohlavi
sex_income = pd.crosstab(df_clean['sex'], df_clean['income'], normalize='index') * 100
sex_income.plot(kind='bar', ax=axes[1], color=['#ff9999', '#99ff99'])
axes[1].set_title('Prijem podle pohlavi (%)', fontsize=14)
axes[1].set_xlabel('Pohlavi')
axes[1].set_ylabel('Procento')
axes[1].legend(['<=50K', '>50K'])
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

print("\nJiz v datech vidime nerovnost - muzi maji vyrazne vyssi sanci na vyssi prijem!")

---

## 4. Trenink modelu

Natrénujeme RandomForest klasifikator a zmerime jeho celkovou presnost.

In [ ]:
# Priprava dat pro model
df_model = df_clean.copy()

# Ulozime si puvodni pohlavi pro pozdejsi audit
sex_original = df_model['sex'].copy()
race_original = df_model['race'].copy() if 'race' in df_model.columns else None

# Encodovani kategorickych promennych
label_encoders = {}
for col in df_model.columns:
    if df_model[col].dtype == 'object':
        le = LabelEncoder()
        df_model[col] = le.fit_transform(df_model[col].astype(str))
        label_encoders[col] = le

# Priprava X a y
X = df_model.drop('income', axis=1)
y = df_model['income']

# Rozdeleni na trenovaci a testovaci data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Ulozime si odpovidajici pohlavi pro testovaci data
test_indices = X_test.index
sex_test = sex_original.loc[test_indices]

print(f"Trenovacich vzorku: {len(X_train)}")
print(f"Testovacich vzorku: {len(X_test)}")

In [ ]:
# Trenink modelu
print("Trenuji RandomForest klasifikator...")
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Predikce
y_pred = model.predict(X_test)

# Celkova presnost
overall_accuracy = accuracy_score(y_test, y_pred)

print("\n" + "=" * 60)
print(f"CELKOVA PRESNOST MODELU: {overall_accuracy*100:.2f}%")
print("=" * 60)
print("\nNa prvni pohled to vypada dobre! Ale je model spravedlivy?")

---

## 5. AUDIT SPRAVEDLNOSTI - Klicova cast!

Ted provedeme **fairness audit** - zmerime, jak si model vede pro ruzne demograficke skupiny.

In [ ]:
# Priprava auditu
audit_df = pd.DataFrame({
    'sex': sex_test.values,
    'true_income': y_test.values,
    'predicted_income': y_pred
})

# Rozdelime podle pohlavi
audit_male = audit_df[audit_df['sex'] == 'Male']
audit_female = audit_df[audit_df['sex'] == 'Female']

# Vypocet presnosti pro kazdou skupinu
accuracy_male = accuracy_score(audit_male['true_income'], audit_male['predicted_income'])
accuracy_female = accuracy_score(audit_female['true_income'], audit_female['predicted_income'])

print("=" * 60)
print("AUDIT SPRAVEDLNOSTI PODLE POHLAVI")
print("=" * 60)
print(f"\nCelkova presnost:     {overall_accuracy*100:.2f}%")
print(f"Presnost pro MUZE:    {accuracy_male*100:.2f}%")
print(f"Presnost pro ZENY:    {accuracy_female*100:.2f}%")
print(f"\nRozdil:               {abs(accuracy_male - accuracy_female)*100:.2f} procentnich bodu")

In [ ]:
# Vizualizace rozdilu
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sloupcovy graf presnosti
skupiny = ['Celkova', 'Muzi', 'Zeny']
presnosti = [overall_accuracy, accuracy_male, accuracy_female]
barvy = ['steelblue', '#4CAF50', '#FF5722']

bars = axes[0].bar(skupiny, [p*100 for p in presnosti], color=barvy)
axes[0].set_ylabel('Presnost (%)', fontsize=12)
axes[0].set_title('Porovnani presnosti modelu', fontsize=14, fontweight='bold')
axes[0].set_ylim(0, 100)

# Pridani hodnot nad sloupce
for bar, p in zip(bars, presnosti):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                f'{p*100:.1f}%', ha='center', fontsize=11)

# Horizontal bar pro porovnani
disparity = abs(accuracy_male - accuracy_female) * 100
axes[1].barh(['Rozdil v presnosti'], [disparity], color='red', alpha=0.7)
axes[1].set_xlim(0, 20)
axes[1].set_xlabel('Procentni body', fontsize=12)
axes[1].set_title(f'Disparita mezi skupinami: {disparity:.2f}%', fontsize=14, fontweight='bold')

# Hranice prijatelnosti
axes[1].axvline(x=5, color='orange', linestyle='--', label='Hranice 5%')
axes[1].axvline(x=10, color='red', linestyle='--', label='Hranice 10%')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Detailnejsi analyza - False Positive a False Negative Rate
def calculate_rates(y_true, y_pred):
    """Vypocita ruzne metriky fairness."""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    # Metriky
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0  # True Positive Rate (Recall)
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0  # False Positive Rate
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0  # False Negative Rate
    
    return {
        'accuracy': accuracy,
        'tpr': tpr,
        'fpr': fpr,
        'fnr': fnr
    }

# Vypocet pro obe skupiny
rates_male = calculate_rates(audit_male['true_income'], audit_male['predicted_income'])
rates_female = calculate_rates(audit_female['true_income'], audit_female['predicted_income'])

print("=" * 70)
print("DETAILNI METRIKY FAIRNESS")
print("=" * 70)
print(f"{'Metrika':<25} {'Muzi':>15} {'Zeny':>15} {'Rozdil':>15}")
print("-" * 70)

for metrika in ['accuracy', 'tpr', 'fpr', 'fnr']:
    m = rates_male[metrika]
    f = rates_female[metrika]
    rozdil = abs(m - f)
    print(f"{metrika.upper():<25} {m*100:>14.2f}% {f*100:>14.2f}% {rozdil*100:>14.2f}%")

print("\n" + "=" * 70)
print("VYSVETLENI METRIK:")
print("-" * 70)
print("TPR (True Positive Rate): Jak casto spravne predikujeme vyssi prijem")
print("FPR (False Positive Rate): Jak casto CHYBNE predikujeme vyssi prijem")
print("FNR (False Negative Rate): Jak casto CHYBNE predikujeme nizsi prijem")

---

## 6. Interpretace vysledku - "Aha!" moment

In [ ]:
# Interpretace
print("=" * 70)
print("INTERPRETACE VYSLEDKU")
print("=" * 70)
print()
print("CO JSME ZJISTILI?")
print("-" * 50)
print()
print("1. CELKOVA PRESNOST vypada dobre (~85%)")
print("   -> Na prvni pohled model funguje!")
print()
print("2. ALE kdyz se podivame na jednotlive skupiny...")
print(f"   -> Pro muze: {accuracy_male*100:.1f}%")
print(f"   -> Pro zeny: {accuracy_female*100:.1f}%")
print(f"   -> Rozdil: {abs(accuracy_male-accuracy_female)*100:.1f} procentnich bodu")
print()
print("3. PROC JE TO PROBLEM?")
print("   - Model SYSTEMATICKY znevyhodnuje jednu skupinu")
print("   - Pri 1000 rozhodnutich = stovky nespravedlivych pripadu")
print("   - Pokud model rozhoduje o uverech/praci = diskriminace")
print()
print("4. KDE SE BIAS VZAL?")
print("   - Z historickych dat - odrazejí spo;ecenske nerovnosti")
print("   - Model se naucil existujici predsudky")
print("   - Neni to chyba AI, ale NASICH DAT")

---

## 7. Reseni - Jak zmernit bias?

In [ ]:
# Mozna reseni
print("=" * 70)
print("JAK ZMERNIT ALGORITMICKOU NESPRAVEDLNOST?")
print("=" * 70)
print()

reseni = [
    ("1. ODSTRANENI CITLIVYCH ATRIBUTU", 
     "Nepoužívat pohlavi, rasu, vek primo v modelu.\n"
     "   PROBLEM: Proxy promenne (PSC, jmeno) mohou stale korelovat."),
    
    ("2. VYVAZENI DATASETU",
     "Oversampling mensinovych skupin, undersampling vetsiny.\n"
     "   Zajisti rovnomerne zastoupeni vsech skupin."),
    
    ("3. FAIRNESS-AWARE ALGORITMY",
     "Specialni algoritmy jako: Fairlearn, AI Fairness 360.\n"
     "   Optimalizuji presnost i spravedlnost zaroven."),
    
    ("4. POST-PROCESSING",
     "Upravit prahy rozhodovani pro ruzne skupiny.\n"
     "   Vyrovnat False Positive/Negative rates."),
    
    ("5. PRAVIDELNY AUDIT",
     "Kontinualne monitorovat metriky fairness.\n"
     "   Odhalit drift a degradaci v case.")
]

for nazev, popis in reseni:
    print(f"\n{nazev}")
    print("-" * 50)
    print(f"   {popis}")

In [ ]:
# Experiment: Model BEZ citlivych atributu
print("\n" + "=" * 70)
print("EXPERIMENT: Model bez pohlavi")
print("=" * 70)

# Odstranime pohlavi z features
if 'sex' in X_train.columns:
    X_train_fair = X_train.drop('sex', axis=1)
    X_test_fair = X_test.drop('sex', axis=1)
    
    # Trenink noveho modelu
    model_fair = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    model_fair.fit(X_train_fair, y_train)
    y_pred_fair = model_fair.predict(X_test_fair)
    
    # Audit
    audit_fair = pd.DataFrame({
        'sex': sex_test.values,
        'true_income': y_test.values,
        'predicted_income': y_pred_fair
    })
    
    acc_fair_male = accuracy_score(
        audit_fair[audit_fair['sex']=='Male']['true_income'],
        audit_fair[audit_fair['sex']=='Male']['predicted_income']
    )
    acc_fair_female = accuracy_score(
        audit_fair[audit_fair['sex']=='Female']['true_income'],
        audit_fair[audit_fair['sex']=='Female']['predicted_income']
    )
    
    print(f"\nPuvodni model:")
    print(f"  Muzi: {accuracy_male*100:.2f}%, Zeny: {accuracy_female*100:.2f}%")
    print(f"  Rozdil: {abs(accuracy_male-accuracy_female)*100:.2f}%")
    print(f"\nModel BEZ pohlavi:")
    print(f"  Muzi: {acc_fair_male*100:.2f}%, Zeny: {acc_fair_female*100:.2f}%")
    print(f"  Rozdil: {abs(acc_fair_male-acc_fair_female)*100:.2f}%")
else:
    print("Sloupec 'sex' neni v datech.")

---

## 8. Mini-kviz

In [ ]:
# Mini-kviz
print("=" * 60)
print("KVIZ: RIZIKA AI A FAIRNESS")
print("=" * 60)
print()

otazky = [
    {
        "otazka": "1. Co je algoritmicky bias?",
        "moznosti": [
            "a) Chyba v kodu",
            "b) Systematicke znevyhodnovani urcitych skupin",
            "c) Pomalost algoritmu",
            "d) Typ neuronove site"
        ],
        "spravna": "b",
        "vysvetleni": "Bias je systematicka nespravedlnost, kdy model znevyhodnuje urcite demograficke skupiny."
    },
    {
        "otazka": "2. Odkud se bias nejcasteji bere?",
        "moznosti": [
            "a) Z hardwaru pocitace",
            "b) Z historickych dat obsahujicich spolecenske nerovnosti",
            "c) Z internetu",
            "d) Z programovaciho jazyka"
        ],
        "spravna": "b",
        "vysvetleni": "Model se uci z dat, ktera casto odrazeji existujici spolecenske predsudky a nerovnosti."
    },
    {
        "otazka": "3. Proc nestaci merit jen celkovou presnost modelu?",
        "moznosti": [
            "a) Celkova presnost je vzdy spatna",
            "b) Muze skryvat velke rozdily mezi skupinami",
            "c) Je prilis slozita na vypocet",
            "d) Nema zadny vyznam"
        ],
        "spravna": "b",
        "vysvetleni": "Vysoka celkova presnost muze maskovat fakt, ze model funguje spatne pro minoritni skupiny."
    },
    {
        "otazka": "4. Co je proxy promenna?",
        "moznosti": [
            "a) Promenna zastupujici jinou, citlivou promennou",
            "b) Skryta vrstva neuronove site",
            "c) Typ databaze",
            "d) Metoda trenovani"
        ],
        "spravna": "a",
        "vysvetleni": "Proxy promenna (napr. PSC) muze korelovat s citlivymi atributy (rasa) a umoznit neprimou diskriminaci."
    },
    {
        "otazka": "5. Co je prvni krok k naprave algoritmicke nespravedlnosti?",
        "moznosti": [
            "a) Vyhodit model",
            "b) Merit a auditovat fairness metriky",
            "c) Pouzit jiny programovaci jazyk",
            "d) Ignorovat problem"
        ],
        "spravna": "b",
        "vysvetleni": "Prvnim krokem je vzdy mereni - bez auditu nevime, ze problem existuje."
    }
]

for q in otazky:
    print(q["otazka"])
    for m in q["moznosti"]:
        print(f"   {m}")
    print(f"   Spravna odpoved: {q['spravna']}")
    print(f"   > {q['vysvetleni']}")
    print()

---

## 9. Shrnuti kapitoly

| Koncept | Popis |
|---------|-------|
| **Algoritmicky bias** | Systematicke znevyhodnovani urcitych skupin modelem |
| **Fairness audit** | Mereni presnosti pro ruzne demograficke skupiny |
| **TPR/FPR/FNR** | Metriky pro hodnoceni spravedlnosti |
| **Proxy promenne** | Promenne neprime korelujici s citlivymi atributy |
| **Fairness-aware ML** | Algoritmy optimalizujici presnost i spravedlnost |

### Klicove poznatky:
- Vysoka celkova presnost nezarucuje spravedlnost
- Bias pochazi z dat, ne z AI samotne
- Mereni je prvni krok k naprave
- Odpovednost za fer AI lezi na nas

---

## 10. Vyzva pro vas

In [ ]:
# VYZVA 1: Audit podle rasy
print("VYZVA 1: Provedte audit podle rasy")
print("="*50)
print()
print("Modifikujte kod auditu tak, aby analyzoval")
print("rozdily v presnosti mezi rasovymi skupinami.")
print()
print("Otazky k zamysleni:")
print("- Jsou rozdily mezi rasami vetsi nez mezi pohlavimi?")
print("- Ktera skupina je nejvice znevyhodnena?")

In [ ]:
# VYZVA 2: Vyzkousejte fairness knihovnu
print("VYZVA 2: Pouzijte knihovnu Fairlearn")
print("="*50)
print()
print("Instalace: pip install fairlearn")
print()
print("Fairlearn nabizi:")
print("- MetricFrame pro audit")
print("- Mitigacni algoritmy")
print("- Vizualizacni nastroje")
print()
print("Dokumentace: https://fairlearn.org/")

In [ ]:
# VYZVA 3: Eticky rozmer
print("VYZVA 3: Zamyslete se nad etickymi otazkami")
print("="*50)
print()
print("1. Mel by se model pouzit pro rozhodovani o uverech?")
print("2. Kdo je zodpovedny za diskriminacni rozhodnuti - vyvojar nebo uzivatel?")
print("3. Je 'fer' mit stejnou presnost pro vsechny skupiny?")
print("4. Jak vyvazit presnost a spravedlnost?")

---

## Dalsi kroky

V pristich kapitolach se naucime:
- Pokrocile techniky mitigace biasu
- Interpretovatelnost modelu (proc model rozhoduje tak, jak rozhoduje)
- Nasazeni AI do produkce zodpovednym zpusobem

**Pamatujte: S velkou moci prichazi velka zodpovednost. Jako tvurci AI mame povinnost budovat systemy, ktere jsou fer pro vsechny!**